# Dummy data generation test area.

Here i'll be testing the code i'll be using to generate dummy data

In [1]:
# Libraries
import numpy as np
import pandas as pd
import yaml
from faker import Faker
from datetime import datetime